In [4]:
# tensorflow imports
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import TimeDistributed

# data structure imports
import pickle
import numpy as np
from dateutil.parser import parse

# define universal model
model = Sequential()
model.add(Bidirectional(LSTM(units=512, return_sequences=True, dropout=0.05), input_shape=(5,11)))
model.add(LSTM(units=256, return_sequences=True, dropout=0.05))
model.add(TimeDistributed(Dense(3)))
model.compile(loss='mse', optimizer='adadelta')

# load weights
model.load_weights('/tf/huraim/bd_lstm_td_universal.h5')

# load scaler
with open('/tf/huraim/feature_scaler.pkl', 'rb') as f:
  scaler = pickle.load(f)

# define helper functions
def feature_extraction(timestep, previous) :
    '''
    PURPOSE: Calculate the features for a machine learning model within the context of hurricane-net
    METHOD: Use the predictors and the calculation methodology defined in Knaff 2013
    INPUT:  timestep - current dictionary of features in the hurricane object format
            previous - previous timestep dictionary of features in the hurricane object format
    OUTPUT: Dictionary of features
    
    timestep = {
      'lat' : float,
      'long' : float,
      'max-wind' : float,
      'entry-time' : datetime
    }
    '''
    features = {
        'lat' : timestep['lat'],
        'long' : timestep['long'],
        'max_wind' : timestep['max_wind'],
        'delta_wind' : (timestep['max_wind'] - previous['max_wind']) / # Calculated from track (12h)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 43200),
        'min_pressure' : timestep['min_pressure'], 
        'zonal_speed' : (timestep['lat'] - previous['lat'])/ # Calculated from track (per hour)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 3600),
        'meridonal_speed' : (timestep['long'] - previous['long'])/# Calculated from track (per hour)
            ((timestep['entry_time'] - previous['entry_time']).total_seconds() / 3600),
        'year' : timestep['entry_time'].year,
        'month' : timestep['entry_time'].month,
        'day' : timestep['entry_time'].day,
        'hour' : timestep['entry_time'].hour,
    }
    return features

def hurricane_ai(input):
  '''
  input = {
    -120 : timestep,
    -96 : timestep,
    -72 : timestep,
    -48 : timestep,
    -24 : timestep,
    0 : timestep
  }
  output = {
    24 : prediction,
    48 : prediction,
    72 : prediction,
    96 : prediction,
    120 : prediction
  }
  timestep = {
      'lat' : float,
      'long' : float,
      'max-wind' : float,
      'min_pressure' : float,
      'entry-time' : datetime
  }
  prediction = {
    'lat' : float,
    'long' : float,
    'max-winds' : float
  }
  '''
  # Take entries and transform them into our data model
  extract = []
  temp = None
  for index, value in enumerate([-120, -96, -72, -48, -24, 0]):
    if not index :
      temp = input[value]
      continue
    else:
      extract.append(list(feature_extraction(input[value], temp).values()))
      temp = input[value]
  
  state = np.expand_dims(scaler.transform(extract), axis = 0)
  print('extract: {}, state: {}'.format(extract, state))
  # Finally, use our hurricane ai to predict storm state
  lat = [output[0] for output in scaler.inverse_transform(
      [[lat[0],0,0,0,0,0,0,0,0,0,0] for lat in model.predict(state)[0]])]
  long = [output[1] for output in scaler.inverse_transform(
      [[0,long[0],0,0,0,0,0,0,0,0,0] for long in model.predict(state)[0]])]
  wind = [output[2] for output in scaler.inverse_transform(
      [[0,0,wind[0],0,0,0,0,0,0,0,0] for wind in model.predict(state)[0]])]
   
  output = dict()
  for index, value in enumerate([24, 48, 72, 96, 120]) :
    output[value] = {
        'lat' : lat[index],
        'long' : long[index],
        'max_wind' : wind[index] * 1.15078
    }
  
  return output

In [5]:
input = {
  0 : {
      'entry_time' : parse('Fri Aug 30 2019 1100 PM'),
      'lat' : 25.5,
      'long' : 71.4,
      'max_wind' : 140 * 0.868976 , # mph to knots
      'min_pressure' : 948.0
    },
  -24 : {
      'entry_time' : parse('Thu Aug 29 2019 1100 PM'),
      'lat' : 23.3,
      'long' : 68.4,
      'max_wind' : 105 * 0.868976 , # mph to knots
      'min_pressure' : 977.0
    },
  -48 : {
      'entry_time' : parse('Wed Aug 28 2019 1100 PM'),
      'lat' : 19.7,
      'long' : 66.0,
      'max_wind' : 85 * 0.868976 , # mph to knots
      'min_pressure' : 986.0
    },
  -72 : {
      'entry_time' : parse('Tue Aug 27 2019 1100 PM'),
      'lat' : 16.0,
      'long' : 63.0,
      'max_wind' : 50 * 0.868976 , # mph to knots
      'min_pressure' : 1006.0
    },
  -96 : {
      'entry_time' : parse('Mon Aug 26 2019 1100 PM'),
      'lat' : 13.2,
      'long' : 59.7,
      'max_wind' : 50 * 0.868976 , # mph to knots
      'min_pressure' : 1003.0
    },
  -120 : {
      'entry_time' : parse('Sun Aug 25 2019 1100 PM'),
      'lat' : 11.7,
      'long' : 55.3,
      'max_wind' : 50 * 0.868976 , # mph to knots
      'min_pressure' : 1003.0
    }
}

In [6]:
hurricane_ai(input)

extract: [[13.2, 59.7, 43.4488, 0.0, 1003.0, 0.0625, 0.18333333333333357, 2019, 8, 26, 23], [16.0, 63.0, 43.4488, 0.0, 1006.0, 0.1166666666666667, 0.13749999999999987, 2019, 8, 27, 23], [19.7, 66.0, 73.86296, 15.207080000000001, 986.0, 0.15416666666666665, 0.125, 2019, 8, 28, 23], [23.3, 68.4, 91.24248, 8.68976, 977.0, 0.15000000000000005, 0.10000000000000024, 2019, 8, 29, 23], [25.5, 71.4, 121.65664, 15.207079999999998, 948.0, 0.09166666666666663, 0.125, 2019, 8, 30, 23]], state: [[[-0.55555556  0.11950791 -0.04432     0.          0.15
    0.15        0.33333333  1.125      -1.          0.5625
    1.13333333]
  [-0.33333333  0.23550088 -0.04432     0.          0.3
    0.8         0.10416667  1.125      -1.          0.625
    1.13333333]
  [-0.03968254  0.34094903  0.824656    1.520708   -0.7
    1.25        0.04166667  1.125      -1.          0.6875
    1.13333333]
  [ 0.24603175  0.42530756  1.32121371  0.868976   -1.15
    1.2        -0.08333333  1.125      -1.          0.75
    1.1

{24: {'lat': 20.341573328524827,
  'long': 56.619663587026295,
  'max_wind': 52.23765487499944},
 48: {'lat': 20.49962975345552,
  'long': 56.97654495919123,
  'max_wind': 52.74289979911539},
 72: {'lat': 20.699284155666827,
  'long': 57.42735192291438,
  'max_wind': 53.38111727960631},
 96: {'lat': 20.87654776200652,
  'long': 57.82760189119726,
  'max_wind': 53.947760093227394},
 120: {'lat': 21.042797775566576,
  'long': 58.20298386625945,
  'max_wind': 54.479196733795845}}